##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

##### Use como base el archivo que provee Tensorflow en su video de "Build an image classifier (ML Zero to Hero - Part 4)"

<p>Termine agregando soporte para GPU en tensorflow, cambie la red neuronal (creo, la cambie varias veces y no se si termine haciendo un circulo completo)</p>

<p> la verdad no se exactamente cuanto cambie, pero si le cambie bastantito</p>

In [ ]:
!nvcc --version

In [25]:
import tensorflow as tf

print(tf.__version__)
print(tf.test.is_built_with_cuda())
print(tf.test.is_built_with_gpu_support())
print("gpu #: ", len(tf.config.list_physical_devices('GPU')))


2.15.0
True
True
gpu #:  4


In [1]:
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()
print('gpu num: {}'.format(strategy.num_replicas_in_sync))


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
gpu num: 1


<h1> Click here if you'd like to modify the basic game functionality</h1> <br> <p>If not, simply run all cells</p>

In [2]:
##The amount of time you need to have your thumb up/down to activate/deactivate the game
thumbs_duration = 2

##Total amount of turns to play against the comupter
total_turns = 5

##The amount of time you have to choose a move.
countdown_time = 5

##The amount of wait between turns. (When it shows you what the computer chose and who won)
wait_between_turns = 2

##The amount of time after a game ends (When it shows you your and the computer scores, and shows who won)
wait_after_game_end=3

<h1>Click here if you'd like to modify the advanced parts of the code</h1>

In [3]:
######Global settings######

#Saved model directory (Already created/trained) 
model_dir = 'rpsmodels'

#Training-validation data split (0.2 means 20% validation, 80% training)
VALIDATION_SPLIT = 0.2


import os

In [4]:
######Training settings.######

#Set to true if you'd like to generate (train) a new model
generate_new_model = True

epochs = 25
steps_per_epoch = 25

##Dataset Input directory
input_dir = r'RPS\rps-custom-sanitized-augmented-v2'

##Dataset 'type'. Is only used for the sake of knowing what dataset type a model was trained with
data_type = 'test'

#The size of the training images. Higher retains more detail but consumes *alot* more memory
image_size  = 150

#The amount of images to use per training step. Higher number = more memory.
batch_size = 512

#If theres validation during training. Recommended to leave at True.s
val=True

#Amount of validation steps after each epoch.
val_step = 3 

In [5]:
#######Augmentation settings######
##These only matter if you also have generate_new_model = True###


#Set to true if you'd like to also generate augmented images.
#If set to true, it will cache the images, if set to false it will augment imaages during training.

make_aug_images = False

#Augmented image output directory.
output_dir = r'RPS\rps-custom-sanitized-augmented-v4'

#Number of workers for image augmentation. Recommended is core count/2
aug_image_workers = int(os.cpu_count()/2)

#Number of augmented images to make per image.
num_augmented_images_per_input = 5

<h2>Model finding</h2>

In [6]:
import os

model_files = []
'rpsmodels/rps-custom11-20231214-25-25-VAL-150-512-5-0.2-sanitized.h5'
try:
    for filename in os.listdir(model_dir):    
        if filename.endswith('.h5'):
            full_path = os.path.join(model_dir, filename)
            creation_date = os.path.getctime(full_path)
            model_number = None
    
            parts = filename.split('-')
            for part in parts:
                if part.startswith('custom'):
                    try:
                        model_number = int(part[6:])  
                    except ValueError:
                        model_number = None
                    break
    
            model_files.append((full_path, creation_date, model_number))
    
    model_files.sort(key=lambda x: x[1])
    sorted_filenames = [item[0] for item in model_files]
    sorted_model_numbers = [item[2] for item in model_files if item[2] is not None]
    
    model_ver= sorted_model_numbers[len(sorted_model_numbers)-1]
    latest_model= sorted_filenames[len(sorted_filenames)-1]
    print("done2")
except:
    print("Model directory not found, creating one.")
    os.makedirs(model_dir, exist_ok=True)

done2


In [7]:
from tensorflow.keras.models import load_model
import sys

try:
    model = load_model(latest_model)
    print("All good, the game should have launched.")
except Exception as e:
    print(f"Model not found. Please introduce one into {model_dir} and rerun the notebook. Error: {e}", file=sys.stderr)
    raise Exception(f'Please introduce a model into {model_dir} and rerun the notebook.')

All good, the game should have launched.


<h2>Image augmentation</h2>

In [8]:
def augment_image(class_name, image_file):
    img_path = os.path.join(input_dir, class_name, image_file)
    img = load_img(img_path)
    img_array = img_to_array(img)

    img_array = img_array[2:-2, 2:-2, :] #adding this line of code solved 4 hours of debugging. Ask me why.

    
    img_array = img_array.reshape((1,) + img_array.shape)

    output_class_dir = os.path.join(output_dir, class_name)
    os.makedirs(output_class_dir, exist_ok=True)

    i = 0
    for batch in datagen.flow(img_array, batch_size=1, save_to_dir=output_class_dir, save_prefix='aug', save_format='jpeg'):
        i += 1
        if i >= num_augmented_images_per_input:
            break

In [9]:
import os
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from multiprocessing import Pool
from augment_image import worker


if(make_aug_images):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=60,
        brightness_range=[0.6, 1.4],  
        channel_shift_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='reflect',
        validation_split=VALIDATION_SPLIT  
    )
    
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
   
    tasks = []
    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name)
        for image_file in os.listdir(class_dir):
            tasks.append((class_name, image_file, input_dir, output_dir, datagen, num_augmented_images_per_input))   
    
    with Pool(processes=aug_image_workers) as pool:  
        pool.map(worker, tasks) 
    
    ##overwrite with a simpler datagen now that the images hae been augmented.
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=VALIDATION_SPLIT
    )
    training_dir = output_dir

else:    
    training_dir = input_dir
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=60,
        brightness_range=[0.6, 1.4],  
        channel_shift_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='reflect',
        validation_split=VALIDATION_SPLIT  
    )
print("done")

done


<h2>Datagen creation</h2>

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_generator = datagen.flow_from_directory(
    training_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  
)

validation_generator = datagen.flow_from_directory(
    training_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  
)

Found 13161 images belonging to 5 classes.
Found 3288 images belonging to 5 classes.


<h2> Model creation</h2>

In [11]:
if (generate_new_model):
    with strategy.scope():
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(image_size, image_size, 3)),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(5, activation='softmax')
        ])
        model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                        

<h2> Model training</h2>

In [ ]:
import datetime


validation_used = 'VAL' if val else 'NOVAL'

if generate_new_model:   
    if (val):
        
        history = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,validation_data = validation_generator, verbose = 1, validation_steps=val_step)
    else:
        history = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch, verbose = 1)

    model_ver= model_ver + 1
    
    current_date = datetime.datetime.now().strftime("%Y%m%d")
    
    os.makedirs(model_dir, exist_ok=True) 


    model_filename = f'rps-custom{model_ver}-{current_date}-{epochs}-{steps_per_epoch}-{validation_used}-{image_size}-{batch_sizee}-{num_augmented_images_per_input}-{VALIDATION_SPLIT}-{data_type}.h5'
    latest_model = os.path.join(model_dir, model_filename)
    
    model.save(latest_model)

print("done")

Epoch 1/25
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
25/25 [==============================] - ETA: 0s - loss: 1.8198 - accuracy: 0.2163

<h2>Model testing</h2>

In [22]:
import matplotlib.pyplot as plt
if(generate_new_model):        
    if(val):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        
        epochs = range(len(acc))
        
        plt.figure(facecolor='#181a1b')
        plt.plot(epochs, acc, 'r', label='Training accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy', color='White')
        plt.tick_params(colors='white')
        plt.legend(loc=0)
        
        ax = plt.gca()
        ax.set_facecolor('#181a1b')
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['left'].set_color('white')
        ax.spines['right'].set_color('white')
        plt.show()
        
        plt.show()
    else:
        acc = history.history['accuracy']
        loss = history.history['loss']
        
        epochs = range(len(acc))
        
        plt.figure(facecolor='#181a1b')
        plt.plot(epochs, acc, 'r', label='Training accuracy')
        plt.title('Training accuracy', color='White')
        plt.tick_params(colors='white')
        plt.legend(loc=0)
        
        ax = plt.gca()
        ax.set_facecolor('#181a1b')
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['left'].set_color('white')
        ax.spines['right'].set_color('white')
        plt.show()
        
        plt.show()
else:
    print("No model generated. Cant retrieve validation data.")

No model generated. Cant retrieve validation data.


<h3>Accuracy as of model rps-custom9-20231213-25-25-VAL-150-128-2-0.2-combined</h3>

<img src="https://i.imgur.com/77eqbyb.png"> 

In [23]:
import sys
import os
import random
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

try:
    model = load_model(latest_model)
except Exception as e:
    print(f"Model not found. Please introduce one into {model_dir} and rerun the notebook. Error: {e}", file=sys.stderr)
    raise Exception(f'Please introduce a model into {model_dir} and rerun the notebook.')


base_dir = 'RPS/rps-custom-validation/'
classes = ['paper', 'rock', 'scissors', 'thumbsdown', 'thumbsup']
print(classes)
print(f'Validation of model {latest_model}')

def load_and_preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(image_size, image_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    return img_array
    
for class_name in classes:
    dir_path = os.path.join(base_dir, class_name)
    file_list = os.listdir(dir_path)
    
    sampled_files = random.sample(file_list, 10)
    
    for file_name in sampled_files:
        file_path = os.path.join(dir_path, file_name)
        
        img_array = load_and_preprocess_image(file_path)
        
        prediction = model.predict(img_array, verbose=0)
        prediction_rounded = np.round(prediction, decimals=2) 
        
        print(f'{file_name}: {prediction_rounded}')

['paper', 'rock', 'scissors', 'thumbsdown', 'thumbsup']
Validation of model rpsmodels\rps-custom11-20231214-25-25-VAL-150-512-5-0.2-sanitized.h5
paper (39).jpg: [[1. 0. 0. 0. 0.]]
paper (141).jpg: [[1. 0. 0. 0. 0.]]
paper (40).jpg: [[1. 0. 0. 0. 0.]]
paper (117).jpg: [[1. 0. 0. 0. 0.]]
paper (115).jpg: [[1. 0. 0. 0. 0.]]
paper (195).jpg: [[0. 1. 0. 0. 0.]]
paper (132).jpg: [[1. 0. 0. 0. 0.]]
paper (86).jpg: [[1. 0. 0. 0. 0.]]
paper (138).jpg: [[1. 0. 0. 0. 0.]]
paper (102).jpg: [[1. 0. 0. 0. 0.]]
rock (109).jpg: [[0. 0. 0. 1. 0.]]
rock (1).jpg: [[0. 1. 0. 0. 0.]]
rock (89).jpg: [[0. 1. 0. 0. 0.]]
rock (88).jpg: [[0. 1. 0. 0. 0.]]
rock (120).jpg: [[0. 1. 0. 0. 0.]]
rock (67).jpg: [[0. 1. 0. 0. 0.]]
rock (112).jpg: [[0. 1. 0. 0. 0.]]
rock (59).jpg: [[0. 1. 0. 0. 0.]]
rock (33).jpg: [[0. 1. 0. 0. 0.]]
rock (184).jpg: [[0. 1. 0. 0. 0.]]
scissors (13).jpg: [[0. 0. 1. 0. 0.]]
scissors (85).jpg: [[0. 0. 1. 0. 0.]]
scissors (93).jpg: [[0. 0. 1. 0. 0.]]
scissors (118).jpg: [[0. 0. 1. 0. 0.]]
sc

<h2> The game itself </h2>

<h3> Pre-requisite functions</h3>

In [36]:
def preprocess_frame(frame):
    frame_rgb=frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #i hate color

    frame_resized = cv2.resize(frame_rgb, (image_size, image_size))

    frame_normalized = frame_resized / 255.0

    frame_expanded = np.expand_dims(frame_normalized, axis=0)

    return frame_expanded, frame_normalized

In [37]:
def determine_winner(user_move, computer_move):
    if user_move == computer_move:
        return 'Draw'
    elif (user_move == 'rock' and computer_move == 'scissors') or \
         (user_move == 'scissors' and computer_move == 'paper') or \
         (user_move == 'paper' and computer_move == 'rock'):
        return 'User wins'
    else:
        return 'Computer wins'

<h3>Initialization data.</h3>

In [41]:
import cv2
import numpy as np
import tensorflow as tf
import time
from IPython.display import clear_output
import tkinter as tk
import subprocess
import pyautogui
import random

moves = ['paper', 'rock', 'scissors']
try:
    model = tf.keras.models.load_model(latest_model)
except Exception as e:
    print(f"Model not found. Please introduce one into {model_dir} and rerun the notebook. Error: {e}", file=sys.stderr)
    raise Exception(f'Please introduce a model into {model_dir} and rerun the notebook.')


x_start = 100
y_start = 100
width = 900
height = 900

width: 2560
height: 1440


<h3> The game itself, for real this time</h3>

In [48]:
start_time = time.time()
prediction_counter = 0
game_active = False
thumbs_up_time = None
thumbs_down_time = None
user_score = computer_score = 0
turn_ammount = 1


while True:

    screen_capture = pyautogui.screenshot(region=(x_start, y_start, width, height))
    frame = np.array(screen_capture)
    frame_to_show = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   
    
    preprocessed_frame, model_sees = preprocess_frame(frame_to_show)
    #cv2.imshow('Model view', model_sees)
    
    prediction = model.predict(preprocessed_frame, verbose=0)
    print(prediction)
    clear_output(wait=True)

    user_move = classes[np.argmax(prediction)]
    cv2.putText(frame_to_show, f'User: {user_move}', (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)


    if user_move == 'thumbsup':
        if thumbs_up_time is None:
            thumbs_up_time = time.time()  
        elif time.time() - thumbs_up_time > thumbs_duration:
            game_active = True
            start_time = time.time()
    else:
        thumbs_up_time = None 

    if user_move == 'thumbsdown':
        start_time = time.time()

        if thumbs_down_time is None:
            thumbs_down_time = time.time() 
        elif time.time() - thumbs_down_time > thumbs_duration:
            game_active = False

    else:
        thumbs_down_time = None  

    if not game_active:            
        cv2.putText(frame_to_show, f'GAME INACTIVE', (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)


    
    if game_active:
        current_time = time.time()
        elapsed_time = current_time - start_time
        countdown_text = str(5 - int(elapsed_time)) 

        cv2.putText(frame_to_show, countdown_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(frame_to_show, f'GAME ACTIVE', (125, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

        
        if elapsed_time > countdown_time:
    
            prediction_counter += 1
    
            computer_move = random.choice(moves)
            
            result = determine_winner(user_move, computer_move)
            if result == "User wins":
                user_score += 1
            elif result == "Computer wins":
                computer_score += 1
            
            cv2.putText(frame_to_show, f'Computer: {computer_move}', (500, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
            if(result == "User wins"):
                cv2.putText(frame_to_show, f'Result: {result}', (175, 650), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 255, 0),5)
            else:
                cv2.putText(frame_to_show, f'Result: {result}', (175, 650), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 0, 255),5)
            
            print(f'Prediction #{prediction_counter}: User: {user_move}, Computer: {computer_move}, Result: {result}')
            #cv2.imshow('Model Input', model_sees)
            cv2.imshow('Screen Capture', frame_to_show)

            cv2.waitKey(wait_between_turns*1000) 
            
            turn_ammount +=1;
            if(turn_ammount > total_turns):
                screen_capture = pyautogui.screenshot(region=(x_start, y_start, width, height))
                framen = np.array(screen_capture)
                frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   

                result_text = f'User: {user_score} | Computer: {computer_score}'


                winner = 'User' if user_score > computer_score else 'Computer' if computer_score > user_score else 'Tie'
                cv2.putText(frame_new, result_text, (195, 450), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)
                cv2.putText(frame_new, f'Winner: {winner}', (195, 550), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)
                cv2.imshow('Screen Capture', frame_new)
                cv2.waitKey(wait_after_game_end*1000)
                game_active = False
                user_score = computer_score = 0
                turn_ammount = 1
                start_time = current_time+wait_after_game_end

                continue
                
            clear_output(wait=True)
            start_time = current_time+wait_between_turns
    cv2.imshow('Screen Capture', frame_to_show)
    cv2.setWindowProperty('Screen Capture', cv2.WND_PROP_TOPMOST, 1)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

[[0.399304   0.01592811 0.27209127 0.28719157 0.02548505]]


<h1>Troubleshooting</h1>

<h3>It says im missing a model</h3>

<p>Make sure you have a model loaded in a folder called 'rpsmodels' in the same directory as this notebook.</p>
<p>Ej, if this notebook is in C:/Rock paper scissors.ipynb then make sure theres a trained model in C:/rpsmodels/model_name.h5</p>
<p>If you dont have one, re-download where this  file game from. It should come with one bundled in.</p>

<h3>My CPU/GPU ramps up and nothing happens</h3>

<p>Make sure to delve into the advanced parts of the code and set generate_new_model and make_aug_images to false.</p>

<h3>Ive waited over 10 minutes and nothing happens.</h3>

<p>Go through the advanced parts of the code, opening each tab until you see red markings which sream errors at you. Scroll to the bottom of them and copy&paste the error into ChatGPT or google.</p>
<p>Its very possible you're simply missing a library.</p>